# 📘 Courseware Builder with Amazon Bedrock, Amazon Nova Premier, and Amazon Nova Canvas

This notebook allows you to:
- Upload a PDF (e.g., textbook, course outline)
- Extract a syllabus using **Amazon Nova Premier**
- Generate **slide-level courseware** and **images** via Amazon Nova Premier and Amazon Nova Canvas
- Compile everything into a **PowerPoint presentation**

### ✅ Requirements
- AWS credentials with Amazon Bedrock access including model access to Amazon Nova Premier and Amazon Nova Canvas
- Python packages: `boto3`, `ipywidgets`, `PyMuPDF`, `python-pptx`, `Pillow`

In [ ]:
!pip install boto3 ipywidgets PyMuPDF python-pptx Pillow

In [ ]:
import boto3
import fitz  # PyMuPDF
import base64
import json
import random
import os
import re
from io import BytesIO
from IPython.display import display, Markdown
import ipywidgets as widgets
from pptx import Presentation
from pptx.util import Inches, Pt

REGION = "us-east-1"
NOVA_PREMIER_ID = "us.amazon.nova-premier-v1:0"
NOVA_CANVAS_ID = "amazon.nova-canvas-v1:0"

bedrock_runtime = boto3.client("bedrock-runtime", region_name=REGION)

In [ ]:
# AWS Authentication with temporary credentials
from getpass import getpass

# Input credentials securely (will not be displayed in the notebook)
aws_access_key = getpass("Enter your AWS Access Key: ")
aws_secret_key = getpass("Enter your AWS Secret Key: ")
aws_session_token = getpass("Enter your AWS Session Token: ")

# Create the client with temporary credentials
bedrock_runtime = boto3.client(
    "bedrock-runtime", 
    region_name=REGION,
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    aws_session_token=aws_session_token
)

In [ ]:
# Test the connection
try:
    # Simple test - list available models
    bedrock = boto3.client(
        'bedrock', 
        region_name=REGION,
        aws_access_key_id=aws_access_key,
        aws_secret_access_key=aws_secret_key,
        aws_session_token=aws_session_token
    )
    response = bedrock.list_foundation_models()
    print("✅ AWS Authentication successful!")
except Exception as e:
    print(f"❌ Authentication error: {e}")

## 📤 Step 1: Upload PDF

In [ ]:
upload_widget = widgets.FileUpload(accept='.pdf', multiple=False)
display(upload_widget)

def save_uploaded_file(upload_widget):
    if not upload_widget.value:
        return None
    
    # Handle tuple type in newer versions of ipywidgets
    if isinstance(upload_widget.value, tuple):
        # Get first file from tuple
        if len(upload_widget.value) > 0:
            file_info = upload_widget.value[0]
            filename = file_info.name
            with open(filename, 'wb') as f:
                f.write(file_info.content)
            return filename
        return None
    else:
        # Handle dictionary type in older versions
        filename = list(upload_widget.value.keys())[0]
        file_info = upload_widget.value[filename]
        with open(filename, 'wb') as f:
            f.write(file_info['content'])
        return filename
    
pdf_path = None

## 🧠 Step 2: Extract Text & Images, Get Syllabus from Nova Premier

In [ ]:
# Run this AFTER file upload
pdf_path = save_uploaded_file(upload_widget)

# Add verification steps
print(f"PDF path: {pdf_path}")

if pdf_path and os.path.exists(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        all_text = ""
        for page in doc:
            all_text += page.get_text()
        
        print(f"Extracted {len(all_text)} characters from PDF")
        print(f"Preview: {all_text[:200]}...")
        
        # Include the actual text content in the prompt
        user_msg = {"role": "user", "content": [{"text": f"""Based on this document text:
        
{all_text[:10000]}

Extract ONLY the actual chapter titles or main topics from this document.
Format as a simple list with no introduction, explanation, or numbering.
Start each item on a new line with a dash."""}]}
        
        response = bedrock_runtime.converse(
            modelId=NOVA_PREMIER_ID,
            messages=[user_msg],
            inferenceConfig={"temperature": 0.2}
        )
        
        syllabus_raw = response['output']['message']['content'][-1]['text']
        
        # Simple cleanup - split by lines and remove any empty lines
        syllabus_items = []
        for line in syllabus_raw.splitlines():
            line = line.strip()
            if line:
                # Remove any leading dashes or bullets
                if line.startswith('-') or line.startswith('•'):
                    line = line[1:].strip()
                syllabus_items.append(line)
        
        display(Markdown("## 📋 Extracted Syllabus"))
        display(Markdown("\n".join(f"- {item}" for item in syllabus_items)))
        
    except Exception as e:
        print(f"Error processing PDF: {e}")
        syllabus_items = ["Error extracting syllabus"]
else:
    print("❌ PDF file not found or not uploaded. Please upload a PDF file first.")
    syllabus_items = ["Sample Topic 1", "Sample Topic 2", "Sample Topic 3"]

## Step 4: Generate Slide Content and Images

In [ ]:
# Step 4: Generate Slide Content and Images
import time
from PIL import Image

slide_contents = []
slide_notes = []  # Add this line to store slide notes
slide_images = []

# Process only a limited number of topics to avoid throttling
max_topics = 5  # Adjust as needed
for i, topic in enumerate(syllabus_items[:max_topics]):
    print(f"Processing topic {i+1}/{min(max_topics, len(syllabus_items))}: {topic}")
    
    # Generate bullet points and notes
    try:
        # Modified prompt to request both bullet points and narrative notes
        user_msg = {"role": "user", "content": [{"text": f"For a slide about '{topic}', provide: 1) 3-5 concise bullet points, and 2) detailed speaker notes that explain each bullet point. Format as 'BULLETS:' followed by bullet points, then 'NOTES:' followed by narrative."}]}
        
        resp = bedrock_runtime.converse(
            modelId=NOVA_PREMIER_ID, 
            messages=[user_msg],
            inferenceConfig={"temperature": 0.2}
        )
        
        response_text = resp['output']['message']['content'][-1]['text']
        
        # Split the response into bullets and notes
        if "BULLETS:" in response_text and "NOTES:" in response_text:
            parts = response_text.split("NOTES:")
            bullets_part = parts[0].replace("BULLETS:", "").strip()
            notes_part = parts[1].strip()
            
            # Extract bullet points with improved filtering
            bullets = []
            for line in bullets_part.splitlines():
                line = line.strip()
                # Skip empty lines and lines that only contain asterisks or bullets
                if line and not line.strip("*• -").strip() == "":
                    # Remove leading bullets, asterisks, and whitespace
                    clean_line = line.strip("•- *")
                    bullets.append(clean_line)
            
            # Filter out any remaining lines that are just asterisks or formatting
            bullets = [b for b in bullets if not all(c in "*• -" for c in b)]
            slide_contents.append(bullets)
            
            # Store notes
            slide_notes.append(notes_part)
        else:
            # Fallback if format isn't followed
            bullets = []
            for line in response_text.splitlines():
                line = line.strip()
                if line and not line.strip("*• -").strip() == "":
                    clean_line = line.strip("•- *")
                    bullets.append(clean_line)
            
            bullets = [b for b in bullets if not all(c in "*• -" for c in b)]
            slide_contents.append(bullets[:5])  # Take first 5 lines as bullets
            slide_notes.append("Speaker notes not available.")
        
        # Wait between API calls to avoid throttling
        time.sleep(3)
    except Exception as e:
        print(f"Error generating content for {topic}: {e}")
        slide_contents.append([f"Key point about {topic}", "Second point", "Third point"])
        slide_notes.append("Speaker notes not available due to an error.")
        time.sleep(5)  # Wait longer after an error

    # Generate image
    try:
        canvas_req = {
            "taskType": "TEXT_IMAGE",
            "textToImageParams": {"text": f"An illustration representing: {topic}. "},
            "imageGenerationConfig": {
                "seed": random.randint(0, 9999999),
                "quality": "standard",
                "width": 512,
                "height": 512,
                "numberOfImages": 1
            }
        }
        
        canvas_resp = bedrock_runtime.invoke_model(
            modelId=NOVA_CANVAS_ID,
            body=json.dumps(canvas_req).encode('utf-8'),
            contentType="application/json"
        )
        
        # Fix for KeyError: 'images'
        response_body = json.loads(canvas_resp['body'].read())
        
        # Check for different response formats
        if 'images' in response_body:
            img_b64 = response_body["images"][0]
            slide_images.append(base64.b64decode(img_b64))
        elif 'image' in response_body:
            img_b64 = response_body["image"]
            slide_images.append(base64.b64decode(img_b64))
        else:
            # Create a placeholder image
            print(f"No image in response for topic: {topic}")
            placeholder = Image.new('RGB', (512, 512), color=(200, 200, 200))
            img_byte_arr = BytesIO()
            placeholder.save(img_byte_arr, format='PNG')
            img_byte_arr.seek(0)
            slide_images.append(img_byte_arr.getvalue())
            
        # Wait between API calls
        time.sleep(3)
    except Exception as e:
        print(f"Error generating image for {topic}: {e}")
        # Create a placeholder image
        placeholder = Image.new('RGB', (512, 512), color=(200, 200, 200))
        img_byte_arr = BytesIO()
        placeholder.save(img_byte_arr, format='PNG')
        img_byte_arr.seek(0)
        slide_images.append(img_byte_arr.getvalue())
        time.sleep(5)  # Wait longer after an error

print(f"Generated content for {len(slide_contents)} topics")

## 🎞️ Step 5: Assemble PowerPoint

In [ ]:
# Step 5: Assemble PowerPoint
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT

# Create a new presentation
prs = Presentation()
blank_layout = prs.slide_layouts[6]  # Blank layout

# For each slide, add bullet points like this:
for i, (topic, bullets, notes, img_data) in enumerate(zip(syllabus_items, slide_contents, slide_notes, slide_images)):
    slide = prs.slides.add_slide(blank_layout)
    
    # Add title
    title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.3), Inches(9), Inches(1))
    title_frame = title_box.text_frame
    title_frame.text = topic
    title_frame.paragraphs[0].font.size = Pt(36)
    
    # Add content with bullet points
    content_box = slide.shapes.add_textbox(Inches(0.5), Inches(1.5), Inches(4.5), Inches(4.5))
    tf = content_box.text_frame
    tf.word_wrap = True
    
    # First paragraph is already created
    p = tf.paragraphs[0]
    p.text = "• " + bullets[0].strip()  # Add bullet manually
    p.font.size = Pt(18)
    p.alignment = PP_PARAGRAPH_ALIGNMENT.LEFT
    p.space_before = Pt(12)
    
    # Add remaining bullet points
    for bullet_text in bullets[1:]:
        p = tf.add_paragraph()
        p.text = "• " + bullet_text.strip()  # Add bullet manually
        p.font.size = Pt(18)
        p.alignment = PP_PARAGRAPH_ALIGNMENT.LEFT
        p.space_before = Pt(6)
    
    # Add image
    try:
        img_stream = BytesIO(img_data)
        slide.shapes.add_picture(img_stream, Inches(5.5), Inches(1.5), height=Inches(4))
    except Exception as e:
        print(f"Error adding image to slide {i+1}: {e}")
    
    # Add notes
    if notes:
        notes_slide = slide.notes_slide
        notes_slide.notes_text_frame.text = notes

# Save presentation with course title in the filename
course_title = syllabus_items[0] if syllabus_items else "Course"
filename = f"output_{course_title.replace(' ', '_')}.pptx"
prs.save(filename)
print(f"✅ PowerPoint saved as '{filename}'")